In [12]:
import re
import spacy
nlp = spacy.load("pl_core_news_sm")

In [27]:
"""
Ważna uwaga:
Pliki z lekturami (trzeba funkcji ustawić argument is_fragment=False) do wczytania muszą mieć następującą strukturę:
Autor

Tytuł(dowolna ilość linii)

Ewentualny numer ISBN

Tekst(dowolna ilość linii)
Ewentualna stopka po treście(przypis od wolnychlektur): -----
"""
"""
Przy podawaniu fragmentu(is_fragment=True) skanowany jest cały plik, więc struktura tekstu jest dowolna
"""
"""
tworzy worek słów z pliku
file_name - nazwa pliku z której utworzyc worek słów; with_stop_words - True/False mówiący czy usuwać stop wordsy czy nie
is_fragment - True/False - mówi czy podawany plik to fragment do sprawdzenia, czy cały utwór
"""
def create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True):
#struktura worka: indeks to indeks, elementem jest lista ze słowem i liczebnością
    bag = {}
    stop_words = []
#tablica znaków interpunkcyjnych do pozbycia się z wyrazów
#TODO: jakieś jeszcze inne znaki?
    punc = '''…!()-[]»{};:'"\,<>./?@#$%^&*_~«—'''+ "\n" + " "
#ewentualne przypisanie stopwordsów
    if not with_stop_words:
        stop_words = get_stop_words_from_file('stopWords.txt')
        
    with open(file_name,'r', encoding='utf-8') as file:
#autor
        if not is_fragment:
            bag[0] = get_author(file)
            
        for line in file:
            for word in line.split():
#jak dotrze do stopki-kończ
                if not is_fragment and word == "-----":
                    return bag
                
            words = words_from_line(line, lemma, punc)
            for word in words:
                word = word.lower()
#jeśli jest stop wordsem- nie uwzględniamy (jeśli w funkcji było True, to 'stop_words' jest puste, czyli nie wejdzie do ifa)    
                if word in stop_words:
                    continue
#dodawanie nowego słowa do worka, jeśli jeszcze go nie ma
                if word not in bag and (len(word) != 0):
                    bag[word] = 1
#inkrementacja liczności słowa, jeśli już istnieje w worku
                elif len(word) != 0:
                    bag[word] += 1
    return bag

def words_from_line(line, lemma, punc):
    words = []
    if not lemma:
        for word in line.split():
            for ele in word:
                if ele in punc:
                    word = word.replace(ele, "")
            words.append(word)
    
    else:
        line = re.sub(' +', ' ', line)
        tokens = nlp(line, disable=["parser", "ner"])
        for token in tokens:
            word = token.lemma_
            if word in punc:
                    continue
            words.append(word)
        
    return words

"""
word_bags - lista stworzonych z lektur worków słów
WAŻNE: w każdym worku pod indeksem 0(liczba, nie string) musi znajdować się autor danego utworu
WAŻNE2: używać przed użyciem funkcji count_percents na workach
struktura outputu: słownik, gdzie kluczem są autorzy, a wartościami worek słów(z których zostanie wyrzucony klucz 0-z autorem)
"""
#TODO: przetetować lepiej
def combine_word_bags(word_bags):
    combined = {}
#przechodzenie po wszystkich workach z listy
    for bag in word_bags:
        keys = combined.keys()
#pobieranie autora z aktualnego worka i usuwanie go ze słownika
        author = bag[0]
        del bag[0]
#jeśli autor już jest w wynikowym worku, to aktualizujemy wystąpienia słów u niego
        if author in keys:
            for elem in bag:
#jeśli autor ma już dane słowo, to dodajemy ilość wystąpień z aktualnie iterowanego worka
                if elem in combined[author]:
                    combined[author][elem] += bag[elem]
#jeśli słowa jeszcze nie ma, to dodajemy rekord
                else:
                    combined[author][elem] = bag[elem]
#jeśli autora nie ma jeszcze w wynikowym worku, to dodajemy rekord, przypisując do niego aktualny worek
        else:
            combined[author] = bag
    return combined

"""
funkcja zmienia ilosć wystąpień danego słowa na procent wartość wystąpień w tekście
"""
def count_percents(word_bag):
    amount = 0
#zliczanie ilości słów w worku
    for elem in word_bag:
        if elem == 0:
            continue
        amount += word_bag[elem]
#przeliczanie wystąpień na wartości procentowe
    for elem in word_bag:
        if elem == 0:
            continue
        word_bag[elem] = word_bag[elem]/amount*100
    return word_bag

In [28]:
#To są różne funkcje pomocnicze
"""
funkcja pobiera stop wordsy z pliku i zwraca je jako listę
"""
def get_stop_words_from_file(file_name):
    stops = []
    borders = ["\\n", "'"]
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            single_line = repr(line)
            for char in borders:
                single_line = single_line.replace(char,"")
            if not(single_line == ""):
                stops.append(single_line)
    return stops

"""
funkcja pobiera autora utworu, przeskakuje tytuł i przeskakuje number ISBN
"""
def get_author(file):
    author = file.readline()
    author = author.replace("\n","")
    author = list(author)
    if author[0] == " ":
        author[0] = ""
    author = "".join(author)
    
    file.readline()
    while not (file.readline()  == "\n"):
        continue
    position_before = file.tell()
    if not file.readline().startswith('ISBN'):
        file.seek(position_before)
    return author

In [29]:
#Do testów TODO: usunąć
b = create_bag_of_words_from_file('testFile.txt', False)
#print(b)
#word_bags = []
#word_bags.append(b)
#combined = combine_word_bags(word_bags)
#count_percents(b)
for i in (b.keys()):
    print("{0} : {1}".format(i, b[i]))
#combined = count_percents(combined)
#print(combined)

0 : Adam Mickiewicz
litwa : 1
ojczyzno : 1
zdrowie : 1
cenić : 1
dowiedzieć : 1
stracić : 1
piękność : 1
twą : 1
ozdoba : 1
widzieć : 1
opisywać : 1
tęsknię : 1


In [30]:
#Do testów TODO: usunąć
b = create_bag_of_words_from_file('testFile.txt', False, lemma=False)
#print(b)
#word_bags = []
#word_bags.append(b)
#combined = combine_word_bags(word_bags)
#count_percents(b)
for i in (b.keys()):
    print("{0} : {1}".format(i, b[i]))
#combined = count_percents(combined)
#print(combined)

0 : Adam Mickiewicz
litwo : 1
ojczyzno : 1
jesteś : 1
zdrowie : 1
cenić : 1
dowie : 1
stracił : 1
piękność : 1
twą : 1
całej : 1
ozdobie : 1
widzę : 1
opisuję : 1
tęsknię : 1
